In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
from os.path import join

import pandas as pd
import numpy as np

import missingno as msno

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
import lightgbm as lgb

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import seaborn as sns

In [ ]:
data_dir = '~/aiffel/kaggle_kakr_housing/data'

train_data_path = join(data_dir, 'train.csv')
sub_data_path = join(data_dir, 'test.csv')      # 테스트, 즉 submission 시 사용할 데이터 경로

print(train_data_path)
print(sub_data_path)

## 1. 데이터 살펴보기
pandas의 read_csv 함수를 사용해 데이터를 읽어오고, 각 변수들이 나타내는 의미를 살펴보겠습니다.
1. ID : 집을 구분하는 번호
2. date : 집을 구매한 날짜
3. price : 타겟 변수인 집의 가격
4. bedrooms : 침실의 수
5. bathrooms : 침실당 화장실 개수
6. sqft_living : 주거 공간의 평방 피트
7. sqft_lot : 부지의 평방 피트
8. floors : 집의 층 수
9. waterfront : 집의 전방에 강이 흐르는지 유무 (a.k.a. 리버뷰)
10. view : 집이 얼마나 좋아 보이는지의 정도
11. condition : 집의 전반적인 상태
12. grade : King County grading 시스템 기준으로 매긴 집의 등급
13. sqft_above : 지하실을 제외한 평방 피트
14. sqft_basement : 지하실의 평방 피트
15. yr_built : 집을 지은 년도
16. yr_renovated : 집을 재건축한 년도
17. zipcode : 우편번호
18. lat : 위도
19. long : 경도
20. sqft_living15 : 2015년 기준 주거 공간의 평방 피트(집을 재건축했다면, 변화가 있을 수 있음)
21. sqft_lot15 : 2015년 기준 부지의 평방 피트(집을 재건축했다면, 변화가 있을 수 있음)

In [ ]:
data = pd.read_csv(train_data_path)
sub = pd.read_csv(sub_data_path)
print('train data dim : {}'.format(data.shape))
print('sub data dim : {}'.format(sub.shape))

price 데이터 분포 확인

In [ ]:
sns.kdeplot(data['price'], fill=True)
plt.xlabel('Price')
plt.ylabel('Density')
plt.title('Distribution of House Prices')
plt.show()

위 그래프가 왼쪽으로 크게 치우쳐 있으므로 로그 변환

In [ ]:
data['price'] = np.log1p(data['price'])

In [ ]:
data.head()

상관관계 파악을 위한 히트맵 확인

In [ ]:
# correlation이 높은 상위 10개의 heatmap
# continuous + sequential variables --> spearman
# abs는 반비례관계도 고려하기 위함
import scipy as sp

cor_abs = abs(data.corr(method='spearman')) 
cor_cols = cor_abs.nlargest(n=10, columns='price').index # price과 correlation이 높은 column 10개 뽑기(내림차순)
# spearman coefficient matrix
cor = np.array(sp.stats.spearmanr(data[cor_cols].values))[0] # 10 x 10
print(cor_cols.values)
plt.figure(figsize=(10,10))
sns.set(font_scale=1.25)
sns.heatmap(cor, fmt='.2f', annot=True, square=True , annot_kws={'size' : 8} ,xticklabels=cor_cols.values, yticklabels=cor_cols.values)

## 2. 간단한 전처리 
각 변수들에 대해 결측 유무를 확인하고, 분포를 확인해보면서 간단하게 전처리를 하겠습니다.
### 결측치 확인
먼저 데이터에 결측치가 있는지를 확인하겠습니다.<br>
missingno 라이브러리의 matrix 함수를 사용하면, 데이터의 결측 상태를 시각화를 통해 살펴볼 수 있습니다.

In [ ]:
msno.matrix(data)

모든 변수에 결측치가 없는 것으로 보이지만, 혹시 모르니 확실하게 살펴보겠습니다.<br>

In [ ]:
for c in data.columns:
    print('{} : {}'.format(c, len(data.loc[pd.isnull(data[c]), c].values)))

price와 sqft_living의 상관관계

In [ ]:
temp_data = pd.concat([data['price'], data['sqft_living']], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.regplot(x='sqft_living', y="price", data=temp_data)

이상치로 보이는 값 확인

In [ ]:
data.loc[data['sqft_living'] > 13000]

이상치 삭제(오히려 성능 떨어져서 복구)

In [ ]:
# data = data.loc[data['id']!=8912]

price와 grade의 상관관계

In [ ]:
temp_data = pd.concat([data['price'], data['grade']], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x='grade', y="price", data=temp_data)

이상치로 보이는 값 확인

In [ ]:
data.loc[(data['price']>12) & (data['grade'] == 3)]

In [ ]:
data.loc[(data['price']>14.7) & (data['grade'] == 8)]

In [ ]:
data.loc[(data['price']>15.5) & (data['grade'] == 11)]

이상치 삭제(오히려 성능 떨어져서 복구)

In [ ]:
# data = data.loc[data['id']!=2302]
# data = data.loc[data['id']!=4123]
# data = data.loc[data['id']!=7173]
# data = data.loc[data['id']!=2775]

price 값을 target으로

In [ ]:
y = data['price']

del data['price']

print(data.columns)

In [ ]:
train_len = len(data)
data = pd.concat((data, sub), axis=0)

print(len(data))
print(train_len)

renovated year을 리모델링 됐는지 안됐는지 one-hot vector로 표현(성능이 떨어져서 삭제)

In [ ]:
# data['is_renovated'] = data['yr_renovated'].apply(lambda x: 0 if x == 0 else 1)
# del data['yr_renovated']

### id, date 변수 정리
id 변수는 모델이 집값을 예측하는데 도움을 주지 않으므로 제거합니다.<br>
date 변수는 연월일시간으로 값을 가지고 있는데, 연월만 고려하는 범주형 변수로 만들겠습니다.

In [ ]:
sub_id = data['id'][train_len:]
del data['id']
data['date'] = data['date'].apply(lambda x : str(x[:6])).astype(int)

In [ ]:
data.head()

### 각 변수들의 분포 확인
한쪽으로 치우친 분포는 모델이 결과를 예측하기에 좋지 않은 영향을 미치므로 다듬어줄 필요가 있습니다.

In [ ]:
fig, ax = plt.subplots(9, 2, figsize=(12, 50))   # 가로스크롤 때문에 그래프 확인이 불편하다면 figsize의 x값을 조절해 보세요. 

# id 변수(count==0인 경우)는 제외하고 분포를 확인합니다.
count = 1
columns = data.columns
for row in range(9):
    for col in range(2):
        sns.kdeplot(data=data[columns[count]], ax=ax[row][col])
        ax[row][col].set_title(columns[count], fontsize=15)
        count += 1
        if count == 19 :
            break

price, bedrooms, sqft_living, sqft_lot, sqft_above, sqft_basement 변수가 한쪽으로 치우친 경향을 보였습니다.<br>
log-scaling을 통해 데이터 분포를 정규분포에 가깝게 만들어 보겠습니다.

In [ ]:
skew_columns = ['bedrooms', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement']

for c in skew_columns:
    data[c] = np.log1p(data[c].values)

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(12, 20))   # 가로스크롤 때문에 그래프 확인이 불편하다면 figsize의 x값을 조절해 보세요. 

count = 0
columns = data.columns
for row in range(4):
    if count >= len(skew_columns):
        break
    for col in range(2):
        sns.kdeplot(data=data[skew_columns[count]], ax=ax[row][col])
        ax[row][col].set_title(skew_columns[count], fontsize=15)
        count += 1
        if count >= len(skew_columns):
            break

어느정도 치우침이 줄어든 분포를 확인할 수 있습니다.

In [ ]:
sub = data.iloc[train_len:, :]
x = data.iloc[:train_len, :]

print(train_len)
print(x.shape)
print(sub.shape)

### PCA를 통해 더 중요한 feature만 남도록 조절<br>
오히려 사용시 오버피팅이 일어나 삭제

In [ ]:
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler

# # 1. 스케일링
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(x)
# X_test_scaled = scaler.transform(sub)

# # 2. PCA
# pca = PCA(n_components=0.9)
# X_pca = pca.fit_transform(X_scaled)
# X_test_pca = pca.transform(X_test_scaled)

# x = X_pca
# sub = X_test_pca

In [ ]:
print(x.shape)
print(sub.shape)

### 여기서부터 학습에 필요한 함수들<br>
첫번째는 rmse 계산 함수

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def rmse(y_test, y_pred):
    return np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(y_pred)))

두번째는 스코어 계산 함수

In [ ]:
def get_scores(models, train, y, include_averaging=False):
    df = {}
    preds_list = []
    
    X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=42)

    for model in models:
        model_name = model.__class__.__name__

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        preds_list.append(y_pred)
        
        df[model_name] = rmse(y_test, y_pred)
        
    if include_averaging:
        y_pred_avg = np.mean(preds_list, axis=0)
        df['AveragingEnsemble'] = rmse(y_test, y_pred_avg)
    
    score_df = pd.DataFrame(df, index=['RMSE']).T.sort_values('RMSE', ascending=False)

    return score_df


다양한 모델에 대해 하이퍼 파라미터를 튜닝하는 함수(random search 사용)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

def random_search(model, params, X, y):
    search = RandomizedSearchCV(
        estimator=model,
        param_distributions=params,
        n_iter=10,
        scoring='neg_root_mean_squared_error',
        cv=5,
        verbose=1,
        random_state=42,
        n_jobs=-1
    )
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    search.fit(X_train, y_train)
    return {
        "model_name": model.__class__.__name__,
        "best_score": -search.best_score_,
        "best_params": search.best_params_,
        "best_estimator": search.best_estimator_
    }

### 앙상블 모델에 사용할 후보들
1. XGBRegressor

In [ ]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, HistGradientBoostingRegressor, ExtraTreesRegressor

scores = []

xgb_model = XGBRegressor(random_state=42)

xgb_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

scores.append(random_search(xgb_model, xgb_params, x, y))

2. LGBMRegressor

In [ ]:
lgb_model = LGBMRegressor(random_state=42)

lgb_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [-1, 5, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [31, 50, 100],
    'subsample': [0.7, 0.8, 1.0]
}

scores.append(random_search(lgb_model, lgb_params, x, y))

3. GradientBoostingRegressor

In [ ]:
gb_model = GradientBoostingRegressor(random_state=42)

gb_params = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 1.0]
}

scores.append(random_search(gb_model, gb_params, x, y))

4. RandomForestRegressor

In [ ]:
rf_model = RandomForestRegressor(random_state=42)

rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'max_features': ['auto', 'sqrt']
}

scores.append(random_search(rf_model, rf_params, x, y))

5. HistGradientBoostingRegressor

In [ ]:
hgb_model = HistGradientBoostingRegressor(random_state=42)

hgb_params = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [5, 10, None],
    'max_iter': [100, 200],
    'l2_regularization': [0.0, 0.1, 1.0]
}

scores.append(random_search(hgb_model, hgb_params, x, y))

6. ExtraTreesRegressor

In [ ]:
et_model = ExtraTreesRegressor(random_state=42)

et_params = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'max_features': ['auto', 'sqrt']
}

scores.append(random_search(et_model, et_params, x, y))

위 모델들 중 가장 RMSE가 낮은 4개의 모델 추출

In [ ]:
results_df = pd.DataFrame(scores)
top_models = results_df.sort_values("best_score").head(4)  # RMSE 낮은 모델 상위 4개
print(top_models)

Averaging을 이용한 성능 비교

In [ ]:
# best_estimator 리스트로부터 모델들 준비
top_models = results_df.sort_values("best_score").head(4)["best_estimator"].tolist()

# Averaging 포함한 성능 비교
score_df = get_scores(top_models, x, y, include_averaging=True)
print(score_df)

학습과 예측값 저장을 모두 포함한 함수<br>
앙상블 기법 중 Averaging, Weighted averaging, Stacking 3가지 기법을 실험함<br>
Stacking에 사용되는 메타모델로 Ridge, SVR, ElasticNet 사용해봄<br>
SVR의 성능이 가장 좋은 것으로 결론지음

In [ ]:
from sklearn.svm import SVR
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import mean_squared_log_error

def save_submission(models, train, y, test, model_name="Ensemble", mode="averaging", weights=None):
    """
    Averaging, Weighted Averaging 또는 Stacking 방식으로 예측 후 submission 파일 저장.

    Parameters:
    - models: base 모델 리스트
    - train, y: 학습 데이터
    - test: 예측용 데이터
    - model_name: 저장될 파일 이름에 포함될 이름
    - mode: 'averaging', 'weighted', 'stacking'
    - weights: weighted averaging 시 사용할 가중치 리스트
    """
    data_dir = os.getenv('HOME') + '/aiffel/kaggle_kakr_housing/data'
    submission_path = join(data_dir, 'sample_submission.csv')
    submission = pd.read_csv(submission_path)

    if mode in ["averaging", "weighted"]:
        predictions = []
        for model in models:
            model.fit(train, y)
            pred = model.predict(test)
            predictions.append(pred)

        preds_array = np.array(predictions)

        if mode == "weighted":
            if weights is None or len(weights) != len(models):
                raise ValueError("weights must be a list of same length as models when mode='weighted'")
            avg_prediction = np.average(preds_array, axis=0, weights=weights)
            y_train_pred = np.average([model.predict(train) for model in models], axis=0, weights=weights)
        else:
            avg_prediction = np.mean(preds_array, axis=0)
            y_train_pred = np.mean([model.predict(train) for model in models], axis=0)

    elif mode == "stacking":
        estimators = [(f"model_{i}", m) for i, m in enumerate(models)]
        meta_model = SVR(C=5.0, epsilon=0.01)

        stack_model = StackingRegressor(
            estimators=estimators,
            final_estimator=meta_model,
            cv=5,
            n_jobs=-1
        )
        stack_model.fit(train, y)
        avg_prediction = stack_model.predict(test)
        y_train_pred = stack_model.predict(train)

    else:
        raise ValueError("mode must be 'averaging', 'weighted' or 'stacking'")

    # 역변환
    avg_prediction = np.expm1(avg_prediction)
    rmsle_score = np.sqrt(mean_squared_log_error(np.expm1(y), np.expm1(y_train_pred)))

    submission['price'] = avg_prediction
    submission_csv_path = f'{data_dir}/submission_{model_name}_{mode}_RMSLE_{rmsle_score:.4f}.csv'
    submission.to_csv(submission_csv_path, index=False)

    print(f'Submission saved to: {submission_csv_path}')

케글 코드를 보던 중 알게 된 2-stage stacking 앙상블 기법 코드(성능저하로 삭제)

In [ ]:
# from sklearn.model_selection import KFold

# def run_2stage_stacking(models, train, y_log, test, sample_submission_path, model_name="2StageStacking", seed=42):
#     """
#     RandomSearch로 튜닝된 모델들로 2-stage stacking 수행 후 제출 파일 저장
    
#     Parameters:
#     - models: best_estimator_ 리스트 (튜닝된 모델들)
#     - train, y_log: 학습 데이터 (y는 np.log1p로 변환된 상태)
#     - test: 예측할 테스트 데이터
#     - sample_submission_path: sample_submission.csv 경로
#     - model_name: 저장할 제출 파일 이름 구분자
#     - seed: 랜덤 시드
#     """
#     oof_preds = []
#     test_preds = []
#     kf = KFold(n_splits=5, shuffle=True, random_state=seed)

#     for i, model in enumerate(models):
#         oof = np.zeros(len(train))
#         test_pred_fold = np.zeros((5, len(test)))  # 5-fold test 예측 저장용

#         for fold, (tr_idx, val_idx) in enumerate(kf.split(train)):
#             X_tr, X_val = train.iloc[tr_idx], train.iloc[val_idx]
#             y_tr, y_val = y_log.iloc[tr_idx], y_log.iloc[val_idx]

#             model.fit(X_tr, y_tr)
#             oof[val_idx] = model.predict(X_val)
#             test_pred_fold[fold] = model.predict(test)

#         oof_preds.append(oof)
#         test_preds.append(np.mean(test_pred_fold, axis=0))

#     # 스택킹 입력 데이터 구성
#     oof_stack = pd.DataFrame(np.array(oof_preds).T, columns=[f'model_{i}' for i in range(len(models))])
#     test_stack = pd.DataFrame(np.array(test_preds).T, columns=[f'model_{i}' for i in range(len(models))])

#     # 메타 모델 학습
#     meta_model = Ridge(alpha=1.0)
#     meta_model.fit(oof_stack, y_log)
#     final_log_pred = meta_model.predict(test_stack)
#     train_log_pred = meta_model.predict(oof_stack)

#     # RMSLE 계산
#     rmsle_score = np.sqrt(mean_squared_log_error(np.expm1(y_log), np.expm1(train_log_pred)))

#     # 제출 저장
#     submission = pd.read_csv(sample_submission_path)
#     submission['price'] = np.expm1(final_log_pred)

#     data_dir = os.path.dirname(sample_submission_path)
#     output_path = f'{data_dir}/submission_{model_name}_RMSLE_{rmsle_score:.4f}.csv'
#     submission.to_csv(output_path, index=False)

#     print(f'[✅] 2-Stage Stacking submission saved to: {output_path}')
#     return output_path

다양한 모델에 대한 앙상블 기법을 활용한 결과값 생성<br>
지금까지 가장 좋은 방식은 4개 모델을 사용하고 메타모델로 SVR을 사용한 Stacking 앙상블 (private 110889.16529)

In [ ]:
top_models = results_df.sort_values("best_score").head(4)["best_estimator"].tolist()

# weights = [0.45, 0.3, 0.25]

save_submission(
    models=top_models,
    train=x,
    y=y,                 # 로그 변환 적용한 y
    test=sub,             # 실제 예측에 쓸 테스트 데이터
    model_name="Top4_svr",
    mode="stacking",
)